# Sequential DQN

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [11]:
import gym
import numpy as np

In [12]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [13]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [14]:
env.reset()
for _ in range(10):
    # env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

state, action, reward, done, info: [-0.04901964  0.23559912  0.04938931 -0.27629412] 1 1.0 False {}
state, action, reward, done, info: [-0.04430766  0.0398086   0.04386343  0.03154809] 0 1.0 False {}
state, action, reward, done, info: [-0.04351149 -0.15591403  0.04449439  0.33774137] 0 1.0 False {}
state, action, reward, done, info: [-0.04662977  0.03854745  0.05124922  0.05941488] 1 1.0 False {}
state, action, reward, done, info: [-0.04585882 -0.15727043  0.05243752  0.36781672] 0 1.0 False {}
state, action, reward, done, info: [-0.04900423 -0.35309675  0.05979385  0.67656202] 0 1.0 False {}
state, action, reward, done, info: [-0.05606616 -0.54899632  0.07332509  0.98745514] 0 1.0 False {}
state, action, reward, done, info: [-0.06704609 -0.74501942  0.0930742   1.30223797] 0 1.0 False {}
state, action, reward, done, info: [-0.08194648 -0.94119075  0.11911895  1.62254565] 0 1.0 False {}
state, action, reward, done, info: [-0.10077029 -0.74765578  0.15156987  1.36923858] 1 1.0 False {}


To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [15]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [16]:
def model_input(state_size, lstm_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    # RNN
    gru = tf.nn.rnn_cell.GRUCell(lstm_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    initial_state = cell.zero_state(batch_size, tf.float32)
    return states, actions, targetQs, cell, initial_state

In [17]:
# RNN generator or sequence generator
def generator(states, initial_state, cell, lstm_size, num_classes, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [18]:
def model_loss(action_size, hidden_size, states, cell, initial_state, actions, targetQs):
    actions_logits, final_state = generator(states=states, cell=cell, initial_state=initial_state, 
                                            lstm_size=hidden_size, num_classes=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    loss = tf.reduce_mean(tf.square(Qs - targetQs))
    return actions_logits, final_state, loss

In [19]:
def model_opt(loss, learning_rate):
    """
    Get optimization operations in order
    :param loss: Generator loss Tensor for action prediction
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # # Optimize
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    # #opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars)

    #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
    grads = tf.gradients(loss, g_vars)
    opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))

    return opt

In [20]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, cell, self.initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, cell=cell, initial_state=self.initial_state)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [21]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), 
#                                size=batch_size, 
#                                replace=False)
#         return [self.buffer[ii] for ii in idx], [self.states[ii] for ii in idx]

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [22]:
# print('state:', np.array(states).shape[1], 
#       'action size: {}'.format((np.max(np.array(actions)) - np.min(np.array(actions)))+1))

In [23]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
action_size = 2
state_size = 4
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
memory_size = 128            # memory capacity - 1000 DQN
batch_size = 128             # experience mini-batch size - 20 DQN
gamma = 0.99                 # future reward discount

In [24]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)


## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [25]:
state = env.reset()
# initial_state = np.zeros([1, hidden_size])
# final_state = np.zeros([1, hidden_size])
for _ in range(batch_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    #memory.states.append([initial_state, final_state])
    state = next_state
    if done is True:
        state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [26]:
# memory.buffer[0], memory.states[0]

In [27]:
# states, rewards, actions

In [ ]:
saver = tf.train.Saver()
episode_rewards_list, rewards_list, loss_list = [], [], []

# TF session for training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        state = env.reset()
        initial_state = sess.run(model.initial_state)

        # Training steps/batches
        while True:
            action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                                  feed_dict = {model.states: state.reshape([1, -1]), 
                                                               model.initial_state: initial_state})
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append([initial_state, final_state])
            total_reward += reward
            initial_state = final_state
            state = next_state

            # Training
            #batch, rnn_states = memory.sample(batch_size)
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            rnn_states = memory.states
            initial_states = np.array([each[0] for each in rnn_states])
            final_states = np.array([each[1] for each in rnn_states])
            next_actions_logits = sess.run(model.actions_logits, 
                                           feed_dict = {model.states: next_states, 
                                                        model.initial_state: final_states[0].reshape([1, -1])})
            nextQs = np.max(next_actions_logits, axis=1) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            loss, _ = sess.run([model.loss, model.opt], feed_dict = {model.states: states, 
                                                                     model.actions: actions,
                                                                     model.targetQs: targetQs,
                                                        model.initial_state: initial_states[0].reshape([1, -1])})
            loss_batch.append(loss)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{}'.format(total_reward),
              'loss:{:.4f}'.format(np.mean(loss_batch)),
              'exploreP:{:.4f}'.format(explore_p))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        loss_list.append([ep, np.mean(loss_batch)])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:58.0000 R:58.0 loss:0.9462 exploreP:0.9943
Episode:1 meanR:37.5000 R:17.0 loss:0.8271 exploreP:0.9926
Episode:2 meanR:30.3333 R:16.0 loss:0.7100 exploreP:0.9910
Episode:3 meanR:32.0000 R:37.0 loss:1.0007 exploreP:0.9874
Episode:4 meanR:30.8000 R:26.0 loss:1.4869 exploreP:0.9849
Episode:5 meanR:28.1667 R:15.0 loss:1.5626 exploreP:0.9834
Episode:6 meanR:25.5714 R:10.0 loss:1.8660 exploreP:0.9824
Episode:7 meanR:25.0000 R:21.0 loss:1.8378 exploreP:0.9804
Episode:8 meanR:23.5556 R:12.0 loss:2.1669 exploreP:0.9792
Episode:9 meanR:23.5000 R:23.0 loss:2.3497 exploreP:0.9770
Episode:10 meanR:24.0000 R:29.0 loss:2.6005 exploreP:0.9742
Episode:11 meanR:23.4167 R:17.0 loss:2.6360 exploreP:0.9726
Episode:12 meanR:22.9231 R:17.0 loss:3.1568 exploreP:0.9709
Episode:13 meanR:23.5000 R:31.0 loss:3.0180 exploreP:0.9680
Episode:14 meanR:23.0000 R:16.0 loss:2.6628 exploreP:0.9664
Episode:15 meanR:23.5000 R:31.0 loss:3.0337 exploreP:0.9635
Episode:16 meanR:22.8235 R:12.0 loss:3.2760 explor

Episode:136 meanR:20.0900 R:19.0 loss:6.5445 exploreP:0.7510
Episode:137 meanR:20.1600 R:16.0 loss:8.9732 exploreP:0.7498
Episode:138 meanR:19.9900 R:11.0 loss:11.4522 exploreP:0.7490
Episode:139 meanR:19.8700 R:13.0 loss:14.0245 exploreP:0.7480
Episode:140 meanR:19.8300 R:18.0 loss:16.2413 exploreP:0.7467
Episode:141 meanR:19.9100 R:24.0 loss:15.9009 exploreP:0.7450
Episode:142 meanR:19.8400 R:11.0 loss:16.5975 exploreP:0.7441
Episode:143 meanR:19.8800 R:21.0 loss:18.1536 exploreP:0.7426
Episode:144 meanR:20.1400 R:39.0 loss:16.2453 exploreP:0.7398
Episode:145 meanR:20.0500 R:13.0 loss:13.8283 exploreP:0.7388
Episode:146 meanR:20.3100 R:37.0 loss:13.5123 exploreP:0.7361
Episode:147 meanR:20.3900 R:30.0 loss:13.1228 exploreP:0.7339
Episode:148 meanR:20.6300 R:35.0 loss:11.8227 exploreP:0.7314
Episode:149 meanR:20.8900 R:39.0 loss:8.7767 exploreP:0.7286
Episode:150 meanR:20.8200 R:23.0 loss:9.1883 exploreP:0.7269
Episode:151 meanR:21.0200 R:42.0 loss:8.8770 exploreP:0.7239
Episode:152 m

Episode:269 meanR:34.7800 R:104.0 loss:8.2292 exploreP:0.4901
Episode:270 meanR:34.9500 R:29.0 loss:14.2891 exploreP:0.4887
Episode:271 meanR:34.8500 R:19.0 loss:13.6637 exploreP:0.4878
Episode:272 meanR:35.1300 R:37.0 loss:17.3314 exploreP:0.4860
Episode:273 meanR:35.2300 R:19.0 loss:23.8897 exploreP:0.4851
Episode:274 meanR:36.2900 R:115.0 loss:14.2140 exploreP:0.4797
Episode:275 meanR:37.0400 R:86.0 loss:8.1489 exploreP:0.4756
Episode:276 meanR:37.4200 R:53.0 loss:10.5380 exploreP:0.4732
Episode:277 meanR:37.3300 R:26.0 loss:11.8645 exploreP:0.4720
Episode:278 meanR:37.3200 R:19.0 loss:16.5027 exploreP:0.4711
Episode:279 meanR:37.4900 R:29.0 loss:20.8529 exploreP:0.4698
Episode:280 meanR:38.0200 R:62.0 loss:20.6774 exploreP:0.4669
Episode:281 meanR:38.6600 R:75.0 loss:14.6266 exploreP:0.4635
Episode:282 meanR:38.9400 R:52.0 loss:13.9695 exploreP:0.4612
Episode:283 meanR:40.4400 R:164.0 loss:8.4188 exploreP:0.4538
Episode:284 meanR:41.8100 R:153.0 loss:7.6141 exploreP:0.4471
Episode:

Episode:401 meanR:102.1700 R:211.0 loss:2.8167 exploreP:0.1509
Episode:402 meanR:103.8000 R:205.0 loss:7.3129 exploreP:0.1480
Episode:403 meanR:107.2700 R:500.0 loss:2.9062 exploreP:0.1413
Episode:404 meanR:106.3600 R:53.0 loss:21.4660 exploreP:0.1406
Episode:405 meanR:105.8600 R:61.0 loss:40.1876 exploreP:0.1398
Episode:406 meanR:106.0800 R:174.0 loss:17.6571 exploreP:0.1376
Episode:407 meanR:108.4000 R:254.0 loss:8.2966 exploreP:0.1344
Episode:408 meanR:110.9700 R:282.0 loss:3.0053 exploreP:0.1309
Episode:409 meanR:112.9100 R:297.0 loss:9.1442 exploreP:0.1274
Episode:410 meanR:113.0300 R:107.0 loss:0.9727 exploreP:0.1261
Episode:411 meanR:113.7000 R:126.0 loss:0.9811 exploreP:0.1247
Episode:412 meanR:114.1500 R:127.0 loss:1.7368 exploreP:0.1232
Episode:413 meanR:114.5100 R:114.0 loss:2.1604 exploreP:0.1219
Episode:414 meanR:117.6300 R:367.0 loss:1.2482 exploreP:0.1179
Episode:415 meanR:118.6800 R:212.0 loss:0.4932 exploreP:0.1156
Episode:416 meanR:118.7400 R:133.0 loss:2.7666 explore

Episode:531 meanR:336.4600 R:500.0 loss:15.8164 exploreP:0.0131
Episode:532 meanR:340.7100 R:500.0 loss:38.8289 exploreP:0.0130
Episode:533 meanR:345.1600 R:500.0 loss:15.4725 exploreP:0.0128
Episode:534 meanR:346.7800 R:500.0 loss:15.2441 exploreP:0.0127
Episode:535 meanR:351.1100 R:500.0 loss:15.2040 exploreP:0.0126
Episode:536 meanR:354.8700 R:500.0 loss:15.0182 exploreP:0.0124
Episode:537 meanR:356.5400 R:500.0 loss:14.5703 exploreP:0.0123
Episode:538 meanR:356.6800 R:500.0 loss:15.3825 exploreP:0.0122
Episode:539 meanR:360.1200 R:500.0 loss:14.6122 exploreP:0.0121
Episode:540 meanR:363.9100 R:500.0 loss:14.2150 exploreP:0.0120
Episode:541 meanR:363.9100 R:500.0 loss:36.8205 exploreP:0.0119
Episode:542 meanR:366.9600 R:500.0 loss:87.9234 exploreP:0.0118
Episode:543 meanR:369.5900 R:500.0 loss:17.1805 exploreP:0.0117
Episode:544 meanR:372.0800 R:500.0 loss:16.3077 exploreP:0.0116
Episode:545 meanR:375.6600 R:500.0 loss:15.5431 exploreP:0.0116
Episode:546 meanR:379.0500 R:500.0 loss:

...

Episode:4352 meanR:338.4000 R:231.0 loss:1.6666 exploreP:0.0100
Episode:4353 meanR:338.4000 R:500.0 loss:1.0870 exploreP:0.0100
Episode:4354 meanR:335.6000 R:220.0 loss:5.2593 exploreP:0.0100
Episode:4355 meanR:335.6000 R:500.0 loss:1.8051 exploreP:0.0100
Episode:4356 meanR:335.6000 R:500.0 loss:17.2004 exploreP:0.0100
Episode:4357 meanR:335.6000 R:500.0 loss:18.1029 exploreP:0.0100
Episode:4358 meanR:335.6000 R:500.0 loss:17.6646 exploreP:0.0100
Episode:4359 meanR:340.4600 R:500.0 loss:19.0946 exploreP:0.0100
Episode:4360 meanR:345.0000 R:500.0 loss:17.0828 exploreP:0.0100
Episode:4361 meanR:349.6000 R:500.0 loss:14.5194 exploreP:0.0100
Episode:4362 meanR:351.2000 R:500.0 loss:17.4620 exploreP:0.0100
Episode:4363 meanR:354.3700 R:337.0 loss:21.9715 exploreP:0.0100
Episode:4364 meanR:355.3500 R:402.0 loss:5.0644 exploreP:0.0100
Episode:4365 meanR:356.0600 R:500.0 loss:6.0751 exploreP:0.0100
Episode:4366 meanR:358.6600 R:500.0 loss:16.5436 exploreP:0.0100
Episode:4367 meanR:355.7100 R:2

Episode:4479 meanR:374.7700 R:187.0 loss:29.7372 exploreP:0.0100
Episode:4480 meanR:374.7700 R:500.0 loss:1.4457 exploreP:0.0100
Episode:4481 meanR:374.7700 R:500.0 loss:19.3854 exploreP:0.0100
Episode:4482 meanR:374.7700 R:500.0 loss:13.8990 exploreP:0.0100
Episode:4483 meanR:372.7700 R:300.0 loss:18.6515 exploreP:0.0100
Episode:4484 meanR:372.7700 R:500.0 loss:0.8614 exploreP:0.0100
Episode:4485 meanR:372.7700 R:500.0 loss:14.1715 exploreP:0.0100
Episode:4486 meanR:367.8800 R:11.0 loss:76.1712 exploreP:0.0100
Episode:4487 meanR:367.8800 R:500.0 loss:16.7158 exploreP:0.0100
Episode:4488 meanR:371.5100 R:500.0 loss:16.9525 exploreP:0.0100
Episode:4489 meanR:371.5100 R:500.0 loss:14.0059 exploreP:0.0100
Episode:4490 meanR:371.5100 R:500.0 loss:14.1149 exploreP:0.0100
Episode:4491 meanR:371.5100 R:500.0 loss:13.5218 exploreP:0.0100
Episode:4492 meanR:376.4000 R:500.0 loss:11.1434 exploreP:0.0100
Episode:4493 meanR:381.3100 R:500.0 loss:18.4031 exploreP:0.0100
Episode:4494 meanR:381.3100 

Episode:4606 meanR:363.9200 R:500.0 loss:16.9206 exploreP:0.0100
Episode:4607 meanR:363.9200 R:500.0 loss:17.8592 exploreP:0.0100
Episode:4608 meanR:363.9200 R:500.0 loss:18.4294 exploreP:0.0100
Episode:4609 meanR:363.9200 R:500.0 loss:15.9843 exploreP:0.0100
Episode:4610 meanR:363.9200 R:500.0 loss:16.7612 exploreP:0.0100
Episode:4611 meanR:363.9200 R:500.0 loss:14.1356 exploreP:0.0100
Episode:4612 meanR:363.9200 R:500.0 loss:15.0923 exploreP:0.0100
Episode:4613 meanR:363.9200 R:500.0 loss:14.1304 exploreP:0.0100
Episode:4614 meanR:363.9200 R:500.0 loss:12.1936 exploreP:0.0100
Episode:4615 meanR:363.9200 R:500.0 loss:12.6861 exploreP:0.0100
Episode:4616 meanR:363.9200 R:500.0 loss:14.6354 exploreP:0.0100
Episode:4617 meanR:363.9200 R:500.0 loss:14.7088 exploreP:0.0100
Episode:4618 meanR:363.9200 R:500.0 loss:15.1176 exploreP:0.0100
Episode:4619 meanR:363.9200 R:500.0 loss:13.1977 exploreP:0.0100
Episode:4620 meanR:360.6300 R:171.0 loss:37.2583 exploreP:0.0100
Episode:4621 meanR:360.63

Episode:4733 meanR:435.6500 R:320.0 loss:24.7820 exploreP:0.0100
Episode:4734 meanR:431.8400 R:119.0 loss:5.8277 exploreP:0.0100
Episode:4735 meanR:432.3900 R:500.0 loss:4.0658 exploreP:0.0100
Episode:4736 meanR:432.3900 R:500.0 loss:20.8308 exploreP:0.0100
Episode:4737 meanR:432.3900 R:500.0 loss:18.7952 exploreP:0.0100
Episode:4738 meanR:432.3900 R:500.0 loss:19.7102 exploreP:0.0100
Episode:4739 meanR:432.3900 R:500.0 loss:18.0881 exploreP:0.0100
Episode:4740 meanR:432.3900 R:500.0 loss:18.6875 exploreP:0.0100
Episode:4741 meanR:432.3900 R:500.0 loss:17.4902 exploreP:0.0100
Episode:4742 meanR:432.3900 R:500.0 loss:17.0257 exploreP:0.0100
Episode:4743 meanR:432.3900 R:500.0 loss:16.6512 exploreP:0.0100
Episode:4744 meanR:432.3900 R:500.0 loss:18.8411 exploreP:0.0100
Episode:4745 meanR:432.3900 R:500.0 loss:15.4282 exploreP:0.0100
Episode:4746 meanR:432.3900 R:500.0 loss:14.9363 exploreP:0.0100
Episode:4747 meanR:432.3900 R:500.0 loss:13.6205 exploreP:0.0100
Episode:4748 meanR:432.3900

Episode:4860 meanR:436.9200 R:500.0 loss:15.1444 exploreP:0.0100
Episode:4861 meanR:436.9200 R:500.0 loss:14.8268 exploreP:0.0100
Episode:4862 meanR:436.9200 R:500.0 loss:15.0960 exploreP:0.0100
Episode:4863 meanR:440.5700 R:500.0 loss:16.6076 exploreP:0.0100
Episode:4864 meanR:444.4400 R:500.0 loss:13.1541 exploreP:0.0100
Episode:4865 meanR:448.2600 R:500.0 loss:16.2203 exploreP:0.0100
Episode:4866 meanR:451.9400 R:500.0 loss:15.6082 exploreP:0.0100
Episode:4867 meanR:456.8300 R:500.0 loss:18.1257 exploreP:0.0100
Episode:4868 meanR:456.8300 R:500.0 loss:17.7690 exploreP:0.0100
Episode:4869 meanR:456.8300 R:500.0 loss:18.7480 exploreP:0.0100
Episode:4870 meanR:452.5600 R:73.0 loss:65.0808 exploreP:0.0100
Episode:4871 meanR:452.5600 R:500.0 loss:11.3506 exploreP:0.0100
Episode:4872 meanR:449.5600 R:200.0 loss:42.1178 exploreP:0.0100
Episode:4873 meanR:444.6700 R:11.0 loss:74.9086 exploreP:0.0100
Episode:4874 meanR:439.7700 R:10.0 loss:103.3289 exploreP:0.0100
Episode:4875 meanR:439.7700

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [184]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model-seq.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
    
    # Episode/epoch
    for _ in range(1):
        state = env.reset()
        total_reward = 0
        
        # Steps/batches
        while True:
            env.render()
            action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                    feed_dict = {model.states: state.reshape([1, -1]), 
                                                                 model.initial_state: initial_state})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # At the end of each episode
        print('total_reward:{}'.format(total_reward))

# Close the env
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward:120.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.